# Demo of an Euler-Maruyama Solver in CUDA

In [ ]:
import torch, math

#### We have the general form for an **Itô Stochastic Differential Equation (SDE)**:

$dX_t=f(t,X_t)dt+g(t,X_t)dW_t$, where:

$X_t \in \mathbb{R}^{d}$: multidimensional state vector

$f(t,X_t)$: drift term, the deterministic evolution

$g(t,X_t)$: diffusion term, the noise intensity

$W_t$: the weiner process (Brownian motion), or a random path with $W_{t+\Delta t} - W_t \sim \mathcal{N}(0, \Delta t)$ 

#### With a **discrete time grid** $t_0,t_1,...,t_N$, we can approximate a solution rather than analytical integration.
The **Euler-Maruyama** method, an extension of the Euler method for ODEs can be described as:

$$X_{k+1} = X_k + f(t_k, X_k)\Delta t + g(t_k, X_k)\Delta W_k$$

In [ ]:
def f(t, x, p): return p["mu"] * x    # drift function
def g(t, x, p): return p["sigma"] * x # diffusion function

def sdeint_euler(f, g, y0, t, params, generator=None):

    y = y0.clone() # initial state
    B, d = y.shape # batch size, state dimension
    T = len(t)     # time domain
    traj = torch.empty((T, B, d), device=y.device, dtype=y.dtype) # preallocate trajectory data
    traj[0] = y    # initial state into tensor

    for k in range(T - 1):
        dt = t[k+1] - t[k]       # time increment
        dW = torch.randn(B, d, device=y.device, generator=generator) * torch.sqrt(dt) # given dW = sqrt(dt) * N(0,1)
        ftk = f(t[k], y, params) # drift increment (deterministic)
        gtk = g(t[k], y, params) # diffusion increment (stochastic)
        y = y + ftk*dt + gtk*dW  # Euler-Maruyama update
        traj[k+1] = y
    return traj


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
B, d = 8192, 1                                      # batch size, state dimension
t = torch.linspace(0, 1, 512, device=device)        # time domain
y0 = torch.ones(B, d, device=device)                # initial state
params = {"mu": 0.2, "sigma": 0.5}                  # SDE parameters
gen = torch.Generator(device=device).manual_seed(0) # random seed

In [13]:
traj = sdeint_euler(f, g, y0, t, params, generator=gen) # solve SDE
xT = traj[-1]                                           # extract final states
mean_emp = xT.mean().item()                             #empirical mean
var_emp = xT.var(unbiased=False).item()                  # empirical variance

In [14]:
mean_th = math.exp(params["mu"] * 1)
var_th = math.exp(2*params["mu"]*1)*(math.exp(params["sigma"]**2*1) - 1)
print(f"mean sim={mean_emp:.4f}, theory={mean_th:.4f}")
print(f"var  sim={var_emp:.4f}, theory={var_th:.4f}")

mean sim=1.2256, theory=1.2214
var  sim=0.4342, theory=0.4237
